# Helpers

In [1]:
from collections import defaultdict, deque
from math import floor, sqrt
from pprint import pprint
from typing import Generator

# Tricks

## 1. Unique Representation of Anagrams (Grouping)

In [2]:
strings: list[str] = ["eat", "tea", "tan", "ate", "nat", "bat"]

### Using List Counts and ASCII Values

In [3]:
def groupgen(strings: list[str]) -> list[list[str]]:
    """Group anagrams."""
    groups: dict[str, list[str]] = defaultdict(list)
    for string in strings:
        counts: list[int] = [0 for _ in range(26)]
        for char in string:
            counts[ord(char) - ord("a")] += 1
        groups["".join(map(str, counts))].append(string)

    return groups

pprint(groupgen(strings))

defaultdict(<class 'list'>,
            {'10000000000001000001000000': ['tan', 'nat'],
             '10001000000000000001000000': ['eat', 'tea', 'ate'],
             '11000000000000000001000000': ['bat']})


### Using k Distinct Primes

In [4]:
def is_prime(num: int) -> bool:
    """Checks whether a number is prime."""
    
    if num % 6 not in {1, 5}:
        return False

    for div in range(2, floor(sqrt(num)) + 1):
        if num % div == 0:
            return False

    return True

def primegen() -> int:
    """Prime number generator."""

    yield from [2, 3]
    num: int = 5
    while True:
        if is_prime(num):
            yield num
        num += 1

# Generate primes
gen: Generator[int, int, int] = primegen()
primes: dict[str, int] = dict(zip("abcdefghijklmnopqrstuvwzyz", [next(gen) for _ in range(26)]))

# Generate groups
groups: dict[int, list[str]] = defaultdict(list)
for string in strings:
    prod: int = 1
    for char in string:
        prod *= primes[char]
    groups[prod].append(string)

pprint(groups)

defaultdict(<class 'list'>,
            {426: ['bat'],
             1562: ['eat', 'tea', 'ate'],
             6106: ['tan', 'nat']})


## 2. MinimumArea Rectangle

In [5]:
points: list[int] = [(1, 1), (1, 3), (3, 1), (3, 3), (2, 2)]

In [6]:
#
#   (1, 3)     (3, 3)
#     *          *
#
#     *          *
#   (1, 1)     (3, 1)
#

def min_rec_area(points: list[tuple[float, float]]) -> float:
    seen: set[tuple[int, int]] = set(map(tuple, points))

    res: float = float("inf")
    for i in range(len(points)):
        p1: tuple[int, int] = points[i]
        # No need to iterate through all points in the second loop
        # as at one point, all of the diagonals will be checked anyway
        for j in range(i):
            p2: tuple[int, int] = points[j]
            # If the points form a diagonal
            if p1[0] != p2[0] and p1[1] != p2[1]:
                # And if we can find another diagonal
                if {(p1[0], p2[1]), (p2[0], p1[1])} <= seen:
                    # Properly update the minimum
                    res = min(res, abs((p1[0] - p2[0]) * (p1[1] - p2[1])))


    # In the end, if `res` has changed, return that value
    # Otherwise, return `0`
    return res if res != float("inf") else 0

pprint(min_rec_area(points))

4


## 3. Balanced Parantheses

The idea here is simple. The problem can be solved by applying the following procedures:

- While there are characters to go through:
    - If we stumble into an opening paranthesis, push it onto the stack
    - If we stumble into a closing paranthesis:
        - Make sure the stack is not empty
        - Make sure that the last item on the stack was the opening one
- Make sure that the stack is empty

In [7]:
string1: str = "((adasdsadl))a{{asld}asl}sd(asdad)()a{asdlaksj{{a}}hl}s()sla(()())das()(asda(asdas))(asd[[]asdas[[kjdsf]aslkdjas][]]asd)d(sadasd)"
string2: str = "((adasdsadl))a{{asld}asl}sd(asdad)()a{asdlaksj{{a}}hl}s()sla(()())das()(asda(asdas))(asd[[]asdas[[kjdsf]aslkdjas][]]asd)d(sadasd)["

In [8]:
def is_balanced(string: str) -> bool:
    """Checks if a string contains balanced sequence of parantheses."""

    stack: list[str] = []
    closing: dict[str, str] = {")": "(", "}": "{", "]": "["}
    opening: set[str] = set(closing.values())
    for char in string:
        if char in opening:
            stack.append(char)
        elif char in closing and (not stack or closing[char] != stack.pop()):
            return False

    return not stack

print(is_balanced(string1))
print(is_balanced(string2))

True
False


## 4. Trie

Implementation of a Trie (aka a prefix tree) data structure.
It is fairly straightforward to implement as a dictionary of dictionaries with a marker specifying the end of a word (if present).

In [9]:
class TrieNode:
    def __init__(self) -> None:
        self._children: dict[str, dict] = {}
        self._isendofword: bool = False

class Trie:
    def __init__(self) -> None:
        self._root: TrieNode = TrieNode()
            
    def insert(self, word: str) -> None:
        """Insert a word into a trie."""

        ptr: TrieNode = self._root
        for char in word:
            if char not in ptr._children:
                ptr._children[char] = TrieNode()
            ptr = ptr._children[char]
            
        ptr._isendofword = True
        
    def search(self, word: str) -> bool:
        """Search for a word in a trie."""
        
        ptr: TrieNode = self._root
        for char in word:
            if char not in ptr._children:
                return Falsse
            ptr = ptr._children[char]
            
        return ptr._isendofword

    def prefix(self, word: str) -> bool:
        """Check if the word is a prefix."""
        
        ptr: TrieNode = self._root
        for char in word:
            if char not in ptr._children:
                return Falsse
            ptr = ptr._children[char]
            
        return True
    
    def __repr__(self) -> str:
        """Breadth-first traversal of a trie."""

        q: list[TrieNode] = deque([self._root])
        res: list[str] = []
        cur: int = 0
        while q:
            level: list[str] = [f"Level {cur}: "]
            for _ in range(len(q)):
                item: TrieNode = q.popleft()
                for char, child in item._children.items():
                    level.append(char)
                    q.append(child)
            res.extend([*level, "\n"])
            cur += 1

        return "".join(res)

In [10]:
words: list[str] = ["caramel", "chocolate", "milk", "ice-cream"]
trie: Trie = Trie()
for word in words:
    trie.insert(word)
    
pprint(trie)

Level 0: cmi
Level 1: ahic
Level 2: role
Level 3: ack-
Level 4: moc
Level 5: elr
Level 6: lae
Level 7: ta
Level 8: em
Level 9: 



## 5. Merge Sort

Merge sort is one of the most efficient sorts out there.
It can also be parallelized (see [this Wikipedia link](https://en.wikipedia.org/wiki/Merge_sort#Parallel_merge_sort))

> Fun fact: Python's built-in `sort` function uses Timsort which is a spionff of merge sort.

In [11]:
def merge(arr1: list[int], arr2: list[int]) -> list[int]:
    """Merge two sorted lists."""
    
    res: list[int] = []
    i: int = 0
    j: int = 0
    while i <= len(arr1) - 1 and j <= len(arr2) - 1:
        if arr1[i] < arr2[j]:
            res.append(arr1[i])
            i += 1
        else:
            res.append(arr2[j])
            j += 1
            
    while i <= len(arr1) - 1:
        res.append(arr1[i])
        i += 1
        
    while j <= len(arr2) - 1:
        res.append(arr2[j])
        j += 1
        
    return res

def merge_sort(arr: list[int]) -> list[int]:
    """Implementes merge sort."""
    
    if len(arr) < 2:
        return arr
    
    mid: int = len(arr) // 2
    left: list[int] = merge_sort(arr[:mid])
    right: list[int] = merge_sort(arr[mid:])
    return merge(left, right)

In [12]:
arr1: list[int] = [1, 2, 4, 5, 6]
arr2: list[int] = [9, 10, 11, 12, 13]
pprint(merge(arr1, arr2))

print()

nums: list[int] = [[2, 1, -1], [10, 2, 123, 123, -123, 123, 213, 5423, 4356, 456454, 6], [123, 0.1, 5.7, 9.1, 10.1]]
pprint(list(map(merge_sort, nums)))

[1, 2, 4, 5, 6, 9, 10, 11, 12, 13]

[[-1, 1, 2],
 [-123, 2, 6, 10, 123, 123, 123, 213, 4356, 5423, 456454],
 [0.1, 5.7, 9.1, 10.1, 123]]


## 

## 6. Selection Sort

One of the worst sorting algorithms out there.
Better than bogosort, but worse than virtually anything else (except for bubble sort).

In [13]:
numbers: list[int] = [3, 2, 5, 56.9, -2, -1, 12.5, 0, 12, 4.2, 6.1]

In [14]:
def selection_sort(nums: list[int]) -> None:
    """Implements bubble sort (in-place implementation)."""

    for i in range(len(numbers)):
        for j in range(i, len(numbers)):
            if numbers[i] > numbers[j]:
                numbers[i], numbers[j] = numbers[j], numbers[i]

In [15]:
pprint(numbers)
selection_sort(numbers)
pprint(numbers)

[3, 2, 5, 56.9, -2, -1, 12.5, 0, 12, 4.2, 6.1]
[-2, -1, 0, 2, 3, 4.2, 5, 6.1, 12, 12.5, 56.9]


## 7. Bubble Sort

In [16]:
numbers: list[int] = [3, 2, 5, 56.9, -2, -1, 12.5, 0, 12, 4.2, 6.1]

In [17]:
def bubble_sort(nums: list[int]) -> None:
    swapped: bool = True
    its: int = 0
    while swapped:
        swapped = False
        for i in range(len(nums) - its - 1):
            if nums[i] > nums[i + 1]:
                nums[i], nums[i + 1] = nums[i + 1], nums[i]
                swapped = True
        its += 1

In [18]:
pprint(numbers)
bubble_sort(numbers)
pprint(numbers)

[3, 2, 5, 56.9, -2, -1, 12.5, 0, 12, 4.2, 6.1]
[-2, -1, 0, 2, 3, 4.2, 5, 6.1, 12, 12.5, 56.9]
